In [23]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Import the CRUD Python module
from animal_sanctuary import AnimalSanctuary

# Database connection details
USER = "aacuser"
PASS = "RighteousFire83!"
HOST = 'nv-desktop-services.apporto.com'
PORT = 31024
DB = "aac"
AUTH_DB = "admin"
COL = "animals"

# Instantiate the AnimalSanctuary class with the connection details
sanctuary = AnimalSanctuary()

# Fetch data from the MongoDB collection
df = pd.DataFrame.from_records(sanctuary.read({}))

# Debug: print information about the DataFrame
print("Columns in the DataFrame:", df.columns.tolist())
print("Number of rows:", len(df))
print("First few rows of data:")
print(df.head())

# Check if '_id' column exists before trying to drop it
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)
    print("'_id' column dropped.")
else:
    print("'_id' column not found in the DataFrame.")

# Define a function to classify rescue types
def classify_rescue_type(breed, sex_upon_outcome, age):
    water_rescue_breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
    mountain_rescue_breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
    disaster_rescue_breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
    
    # Check for exact match in water rescue breeds
    if breed in water_rescue_breeds and "Intact Female" in sex_upon_outcome and 26 <= age <= 156:
        return "Water Rescue"
    # Check for exact match in mountain rescue breeds
    elif breed in mountain_rescue_breeds and "Intact Male" in sex_upon_outcome and 26 <= age <= 156:
        return "Mountain or Wilderness Rescue"
    # Check for exact match in disaster rescue breeds
    elif breed in disaster_rescue_breeds and "Intact Male" in sex_upon_outcome and 20 <= age <= 300:
        return "Disaster or Individual Tracking"
    else:
        return "Other"

# Apply the classification function to the DataFrame
df['rescue_type'] = df.apply(lambda row: classify_rescue_type(row['breed'], row['sex_upon_outcome'], row['age_upon_outcome_in_weeks']), axis=1)

# Dashboard Layout / View
app = JupyterDash('Grazioso Salvare Rescue Animal Dashboard')

app.layout = html.Div([
    html.Center(html.A([
        html.Img(src='/assets/GraziosoSalvareLogo.png', height='100px')
    ], href='https://www.snhu.edu', target='_blank')),
    html.Center(html.B(html.H1('Grazioso Salvare Rescue Animal Search'))),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Div("Designed By: Adam Vosburg", style={'marginBottom': '10px'})),
    html.Div([
        html.Span(f"Logged in User: aacuser", style={'marginRight': '20px'})
    ], style={'marginBottom': '10px', 'marginLeft': '10px'}),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='rescue-type-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'
        ),
    ], style={'textAlign': 'center', 'backgroundColor': '#007bff', 'padding': '10px'}),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
            'whiteSpace': 'normal'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',
        },
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
    ),
    html.Br(),
    html.Hr(),
    html.Center(html.Img(src='/assets/GraziosoSalvareLogo.png', height='100px')),
    html.Div([
        html.Span(f"Logged in User: aacuser", style={'marginRight': '20px'})
    ], style={'marginTop': '10px', 'marginLeft': '10px', 'marginBottom': '10px'}),
        html.Hr(),  
    html.Div([
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top'}
        ),
        html.Div([
            dcc.Graph(id='pie-chart')
        ], style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'top'})
    ]),
])

@app.callback(
    Output('datatable-id', 'data'),
    Input('rescue-type-filter', 'value')
)
def update_dashboard(selected_rescue_type):
    if selected_rescue_type == 'Reset':
        return df.to_dict('records')
    filtered_df = df[df['rescue_type'] == selected_rescue_type]
    return filtered_df.to_dict('records')

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return [html.Div("No data available")]

    dff = pd.DataFrame.from_dict(viewData)

    if not selected_rows:
        row = 0
    else:
        row = selected_rows[0]

    return [
        dl.Map(style={'width': '100%', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
                      children=[
                          dl.Tooltip(dff.iloc[row]['name']),
                          dl.Popup([
                              html.H3(f"Name: {dff.iloc[row]['name']}"),
                              html.P(f"Age: {dff.iloc[row]['age_upon_outcome']}"),
                              html.P(f"Sex: {dff.iloc[row]['sex_upon_outcome']}"),
                              html.P(f"Rescue Category: {dff.iloc[row]['rescue_type']}")
                          ])
                      ])
        ])
    ]

@app.callback(
    Output('pie-chart', 'figure'),
    [Input('rescue-type-filter', 'value'),
     Input('datatable-id', "derived_virtual_data")]
)
def update_pie_chart(selected_rescue_type, viewData):
    if not viewData:
        return px.pie(title="No data available")

    dff = pd.DataFrame.from_dict(viewData)
    
    if selected_rescue_type != 'Reset':
        dff = dff[dff['rescue_type'] == selected_rescue_type]
        title = f'Breed Distribution for {selected_rescue_type}'
    else:
        title = 'Breed Distribution for Selected Rescue Types'

    # Count breeds within the selected rescue type
    breed_counts = dff['breed'].value_counts()
    total_count = breed_counts.sum()

    # Define a threshold for grouping rare breeds into "Other"
    top_n = 10  # Top N breeds to display
    breed_percentages = (breed_counts / total_count * 100).round(2)
    top_breeds = breed_percentages.nlargest(top_n).index
    breeds_to_group = breed_percentages.index.difference(top_breeds)

    # Replace rare breeds with "Other" and calculate new counts
    dff['breed'] = dff['breed'].apply(lambda x: 'Other' if x in breeds_to_group else x)
    breed_counts_grouped = dff['breed'].value_counts()
    breed_percentages_grouped = (breed_counts_grouped / total_count * 100).round(2)

    # Create the pie chart
    fig = px.pie(
        values=breed_percentages_grouped.values,
        names=breed_percentages_grouped.index,
        title=title
    )

    fig.update_layout(
        height=600,
        margin={'l': 20, 'r': 20, 't': 60, 'b': 20},
        title_x=0.7
    )

    # Add detailed information to hover text
    hover_text = [f"{breed}<br>{percentage:.2f}%<br>Count: {count}" 
                  for breed, percentage, count in zip(breed_percentages_grouped.index, breed_percentages_grouped.values, breed_counts_grouped)]
    
    fig.update_traces(hoverinfo='text', text=hover_text)

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


MongoDB connection established successfully
Attempting to find documents with query: {}
Found 9971 documents
Columns in the DataFrame: ['_id', 'rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color', 'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long', 'age_upon_outcome_in_weeks']
Number of rows: 9971
First few rows of data:
                        _id  rec_num age_upon_outcome animal_id animal_type  \
0  66888be1dea8bc977e5275c7        2           1 year   A725717         Cat   
1  66888be1dea8bc977e5275c8        4         7 months   A733653         Cat   
2  66888be1dea8bc977e5275c9        1          3 years   A746874         Cat   
3  66888be1dea8bc977e5275ca        6          5 years   A696004         Dog   
4  66888be1dea8bc977e5275cb        7          2 years   A673830         Dog   

                      breed         color date_of_birth             datetime  \
0    Domestic